In [ ]:
from controller import Robot

In [ ]:
robot = Robot()

In [ ]:
timestep = int(robot.getBasicTimeStep())

In [ ]:
MAX_SPEED = 5.24
MAX_SENSOR_NUMBER = 16
DELAY = 70
MAX_SENSOR_VALUE = 1024
MIN_DISTANCE = 1.0
WHEEL_WEIGHT_THRESHOLD = 100

In [ ]:
wheel_weight = [[150,0], [200,0], [300,0], [600,0], [0,600], [0,300], [0,200], [0,150],
                [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0]]

In [ ]:
left_wheel = robot.getMotor('left wheel')
left_wheel.setPosition(float('+inf'))
left_wheel.setVelocity(0)
right_wheel = robot.getMotor('right wheel')
right_wheel.setPosition(float('+inf'))
right_wheel.setVelocity(0)

In [ ]:
red_led = [robot.getLED('red led 1'), 
           robot.getLED('red led 2'), 
           robot.getLED('red led 3')]

In [ ]:
sensors = []
for i in range(MAX_SENSOR_NUMBER):
    so = robot.getDistanceSensor('so%d' % i)
    so.enable(timestep)
    sensors.append(so)

In [ ]:
FORWARD = 1
LEFT = 2
RIGHT = 3

In [ ]:
state = FORWARD

In [ ]:
seconds = 3
steps = int(seconds * 1000 / timestep)
for _ in range(steps):
    robot.step(timestep)
    speed = [0, 0]
    wheel_weight_total = [0, 0]
    for i in range(MAX_SENSOR_NUMBER):
        sensor_value = sensors[i].getValue()
        if sensor_value == 0.0:
            speed_modifier = 0.0
        else:
            distance = 5.0 * (1.0 - sensor_value / MAX_SENSOR_VALUE)
            if distance < MIN_DISTANCE:
                speed_modifier = 1 - distance / MIN_DISTANCE
            else:
                speed_modifier = 0.0
        wheel_weight_total[0] += wheel_weight[i][0] * speed_modifier
        wheel_weight_total[1] += wheel_weight[i][1] * speed_modifier
    if state == FORWARD:
        if wheel_weight_total[0] > WHEEL_WEIGHT_THRESHOLD:
            speed[0] = 0.7 * MAX_SPEED
            speed[1] = -0.7 * MAX_SPEED
            state = LEFT
        elif wheel_weight_total[1] > WHEEL_WEIGHT_THRESHOLD:
            speed[0] = -0.7 * MAX_SPEED
            speed[1] = 0.7 * MAX_SPEED
            state = RIGHT
        else:
            speed[0] = MAX_SPEED
            speed[1] = MAX_SPEED
    elif state == LEFT:
        if wheel_weight_total[0] > WHEEL_WEIGHT_THRESHOLD or wheel_weight_total[1] > WHEEL_WEIGHT_THRESHOLD:
            speed[0] = 0.7 * MAX_SPEED
            speed[1] = -0.7 * MAX_SPEED
        else:
            speed[0] = MAX_SPEED
            speed[1] = MAX_SPEED
            state = FORWARD
    elif state == RIGHT:
        if wheel_weight_total[0] > WHEEL_WEIGHT_THRESHOLD or wheel_weight_total[1] > WHEEL_WEIGHT_THRESHOLD:
            speed[0] = -0.7 * MAX_SPEED
            speed[1] = 0.7 * MAX_SPEED
        else:
            speed[0] = MAX_SPEED
            speed[1] = MAX_SPEED
            state = FORWARD
    left_wheel.setVelocity(speed[0])
    right_wheel.setVelocity(speed[1])